In [35]:
# Python 2 & 3 Compatibility
from __future__ import print_function, division

# Necessary imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import seaborn as sns
from seaborn import plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

%matplotlib inline


In [36]:
import pickle

# 1. Data Import

In [106]:
# Open the prepped parcel dataframe

with open("../data/processed/0202_mpls_parcel_data_abr.pkl", 'rb') as picklefile: 
    parcel_df = pickle.load(picklefile)
parcel_df.head()

,PIN,USE1_DESC,EMV_TOTAL,TAX_CAPAC,TOTAL_TAX,FIN_SQ_FT,YEAR_BUILT
1,1311821120005,Commercial,0.0,0.0,0.0,0,0
12,3502924230001,Commercial,0.0,0.0,0.0,0,0
13,0202824230266,NaN,0.0,0.0,0.0,0,0
14,0902924420203,Residential,500.0,6.0,9.0,0,0
15,0902924420198,Residential,61500.0,615.0,978.0,0,2001


In [137]:
parcel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129719 entries, 1 to 429557
Data columns (total 7 columns):
PIN           129719 non-null object
USE1_DESC     129718 non-null object
EMV_TOTAL     129719 non-null float64
TAX_CAPAC     129719 non-null float64
TOTAL_TAX     129719 non-null float64
FIN_SQ_FT     129719 non-null int64
YEAR_BUILT    129719 non-null int64
dtypes: float64(3), int64(2), object(2)
memory usage: 7.9+ MB


In [108]:
# Open the private ordinance csv

private_df = pd.read_csv('../data/raw/wcmsp-185935.csv')
cols = ['address', 'postal_code', 'energy_star_score', 'primary_property_type',
       'building_floor_area', 'parking_floor_area',
       'year_built', 'total_ghg_emissions',
       'site_eui', 'weather_normalized_site_eui',
       'source_eui', 'weather_normalized_source_eui',
       'water_use', 'unnamed']

# Rename the columns
private_df.columns = cols
private_df.head()

,address,postal_code,energy_star_score,primary_property_type,building_floor_area,parking_floor_area,year_built,total_ghg_emissions,site_eui,weather_normalized_site_eui,source_eui,weather_normalized_source_eui,water_use,unnamed
0,201 Chicago Ave S,55415,36,Office,62400,0,2012,971,115,118,251,252,708,NaN
1,1011 Nicollet Mall,55403,68,Office,59479,0,1920,986,90,93,256,266,882,NaN
2,4701 Beard Ave S,55410,0,Other,59172,0,1914,1622,157,160,427,428,163,NaN
3,8 60Th St W,55419,76,Worship Facility,60640,62000,1959,268,50,54,77,81,674,NaN
4,700 10Th Ave S,55415,89,Office,60528,0,1920,504,74,77,136,140,414,NaN


In [122]:
priv_df = private_df[['address','energy_star_score',
       'building_floor_area', 'parking_floor_area','total_ghg_emissions',
       'site_eui', 'weather_normalized_site_eui',
       'source_eui', 'weather_normalized_source_eui',
       'water_use']]

In [123]:
priv_df

,address,energy_star_score,building_floor_area,parking_floor_area,total_ghg_emissions,site_eui,weather_normalized_site_eui,source_eui,weather_normalized_source_eui,water_use
0,201 Chicago Ave S,36,62400,0,971,115,118,251,252,708
1,1011 Nicollet Mall,68,59479,0,986,90,93,256,266,882
2,4701 Beard Ave S,0,59172,0,1622,157,160,427,428,163
3,8 60Th St W,76,60640,62000,268,50,54,77,81,674
4,700 10Th Ave S,89,60528,0,504,74,77,136,140,414
5,805 2Nd Ave S,31,70000,18800,575,77,84,138,144,419
6,123 3Rd St N,92,182374,16696,1629,56,58,141,140,2155
7,700 Humboldt Ave N,0,58000,0,515,81,89,148,156,2089
8,700 3Rd St S,63,68047,11000,602,89,96,150,157,236
9,525 West 54th St,4,85345,60000,781,114,124,162,173,747


In [113]:
# Open the prepped property info dataframe

with open("../data/processed/0201_prop_info_data_abr.pkl", 'rb') as picklefile: 
    prop_info_df = pickle.load(picklefile)
prop_info_df.head()

,address,property_id
102824110123,2730 31St St E,102824110123
302824210283,3106 Nicollet Ave,302824210283
302824240197,3335 Blaisdell Ave,302824240197
302924210025,3660 Technology Dr,302924210025
402824210016,1455 Lake St W,402824210016


In [114]:
# Convert the property_id from and into to a string for joining

prop_info_df['property_id_str'] = prop_info_df['property_id'].astype(str)


In [115]:
prop_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 838 entries, 102824110123 to 3602924430088
Data columns (total 3 columns):
address            838 non-null object
property_id        838 non-null int64
property_id_str    838 non-null object
dtypes: int64(1), object(2)
memory usage: 26.2+ KB


In [124]:
# Join the private ordinance data and the property info data

private_prop_info_df = pd.merge(priv_df, prop_info_df, how='inner', left_on='address', right_on='address')

In [125]:
private_prop_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 839 entries, 0 to 838
Data columns (total 12 columns):
address                          839 non-null object
energy_star_score                839 non-null int64
building_floor_area              839 non-null int64
parking_floor_area               839 non-null int64
total_ghg_emissions              839 non-null int64
site_eui                         839 non-null int64
weather_normalized_site_eui      839 non-null int64
source_eui                       839 non-null int64
weather_normalized_source_eui    839 non-null int64
water_use                        839 non-null int64
property_id                      839 non-null int64
property_id_str                  839 non-null object
dtypes: int64(10), object(2)
memory usage: 85.2+ KB


In [161]:
# Join the private and property info data with the parcel data

join_df = pd.merge(private_prop_info_df, parcel_df, how='inner', left_on='property_id_str', right_on='PIN')

In [162]:
join_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 752 entries, 0 to 751
Data columns (total 19 columns):
address                          752 non-null object
energy_star_score                752 non-null int64
building_floor_area              752 non-null int64
parking_floor_area               752 non-null int64
total_ghg_emissions              752 non-null int64
site_eui                         752 non-null int64
weather_normalized_site_eui      752 non-null int64
source_eui                       752 non-null int64
weather_normalized_source_eui    752 non-null int64
water_use                        752 non-null int64
property_id                      752 non-null int64
property_id_str                  752 non-null object
PIN                              752 non-null object
USE1_DESC                        752 non-null object
EMV_TOTAL                        752 non-null float64
TAX_CAPAC                        752 non-null float64
TOTAL_TAX                        752 non-null float64
F

In [163]:
# Drop remaining object fields

join_df.drop(['property_id_str','address', 'PIN'], axis=1, inplace=True)

In [164]:
join_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 752 entries, 0 to 751
Data columns (total 16 columns):
energy_star_score                752 non-null int64
building_floor_area              752 non-null int64
parking_floor_area               752 non-null int64
total_ghg_emissions              752 non-null int64
site_eui                         752 non-null int64
weather_normalized_site_eui      752 non-null int64
source_eui                       752 non-null int64
weather_normalized_source_eui    752 non-null int64
water_use                        752 non-null int64
property_id                      752 non-null int64
USE1_DESC                        752 non-null object
EMV_TOTAL                        752 non-null float64
TAX_CAPAC                        752 non-null float64
TOTAL_TAX                        752 non-null float64
FIN_SQ_FT                        752 non-null int64
YEAR_BUILT                       752 non-null int64
dtypes: float64(3), int64(12), object(1)
memory usage: 9

In [165]:
# Add calculated field

join_df['est_val_per_sqft'] = join_df['EMV_TOTAL'] / join_df['building_floor_area']

In [166]:
# Remove all records where the energy store score is 0

join_df2 = join_df[join_df['energy_star_score'] != 0]

In [167]:
join_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 569 entries, 0 to 743
Data columns (total 17 columns):
energy_star_score                569 non-null int64
building_floor_area              569 non-null int64
parking_floor_area               569 non-null int64
total_ghg_emissions              569 non-null int64
site_eui                         569 non-null int64
weather_normalized_site_eui      569 non-null int64
source_eui                       569 non-null int64
weather_normalized_source_eui    569 non-null int64
water_use                        569 non-null int64
property_id                      569 non-null int64
USE1_DESC                        569 non-null object
EMV_TOTAL                        569 non-null float64
TAX_CAPAC                        569 non-null float64
TOTAL_TAX                        569 non-null float64
FIN_SQ_FT                        569 non-null int64
YEAR_BUILT                       569 non-null int64
est_val_per_sqft                 569 non-null float64
dt

In [193]:
join_df2.USE1_DESC.value_counts()

Condo Garage/Miscellaneous    194
Condominium                   155
Commercial                    143
Cooperative                    25
Residential                    16
 Commercial                    16
Apartment                       6
Double Bungalow                 6
Industrial                      4
Townhouse                       1
 Low Income > 3 Units           1
Common Area (No Value)          1
 Apartment                      1
Name: USE1_DESC, dtype: int64

In [194]:
# Strip white space from the USE1_DESC entries

def strip_entries(df):
    strip = lambda x : x.strip()
    df['USE1_DESC'] = df['USE1_DESC'].apply(strip)

In [195]:
strip_entries(join_df2)
join_df2['USE1_DESC'].value_counts()

/Users/rosiehoyem/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


Condo Garage/Miscellaneous    194
Commercial                    159
Condominium                   155
Cooperative                    25
Residential                    16
Apartment                       7
Double Bungalow                 6
Industrial                      4
Townhouse                       1
Low Income > 3 Units            1
Common Area (No Value)          1
Name: USE1_DESC, dtype: int64

In [197]:
# Remove all residential entries
# TODO: Identify why they ended up the dataset?

join_df3 = join_df2[(join_df2['USE1_DESC'] != "Condominium")]
join_df4 = join_df3[(join_df3['USE1_DESC'] != "Residential")]
join_df5 = join_df4[(join_df4['USE1_DESC'] != "Apartment")]
df_non_res = join_df5[(join_df5['USE1_DESC'] != "Townhouse")]

df_non_res.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 390 entries, 0 to 743
Data columns (total 17 columns):
energy_star_score                390 non-null int64
building_floor_area              390 non-null int64
parking_floor_area               390 non-null int64
total_ghg_emissions              390 non-null int64
site_eui                         390 non-null int64
weather_normalized_site_eui      390 non-null int64
source_eui                       390 non-null int64
weather_normalized_source_eui    390 non-null int64
water_use                        390 non-null int64
property_id                      390 non-null int64
USE1_DESC                        390 non-null object
EMV_TOTAL                        390 non-null float64
TAX_CAPAC                        390 non-null float64
TOTAL_TAX                        390 non-null float64
FIN_SQ_FT                        390 non-null int64
YEAR_BUILT                       390 non-null int64
est_val_per_sqft                 390 non-null float64
dt

In [198]:
# Set misc random USE1_DESC entries to 'Other'

def set_use_to_other(df):
    df['USE1_DESC'].replace('Common Area (No Value)','Other', inplace=True)
    df['USE1_DESC'].replace('Low Income > 3 Units','Other', inplace=True)
    df['USE1_DESC'].replace('Double Bungalow','Other', inplace=True)

In [199]:
set_use_to_other(df_non_res)

/Users/rosiehoyem/anaconda2/envs/py35/lib/python3.5/site-packages/pandas/core/generic.py:3547: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [200]:
df_non_res.USE1_DESC.value_counts()

Condo Garage/Miscellaneous    194
Commercial                    159
Cooperative                    25
Other                           8
Industrial                      4
Name: USE1_DESC, dtype: int64

In [203]:
with open('../data/processed/0202_all_data.pkl', 'wb') as picklefile:
    pickle.dump(df_non_res, picklefile)